# Uma introdução à inferência com BERT

Fonte:
* [Hugging Face Course](https://huggingface.co/course/chapter2/1?fw=pt)
* [An introduction to inference with BERT](https://github.com/BramVanroy/bert-for-inference/blob/master/introduction-to-bert.ipynb)
* [Introdução ao BERT - Gerando sentence embeddings](https://github.com/HAILab-PUCPR/introducao-bert/blob/main/introducao-bert.ipynb)

Esse notebook mostra exemplos de como usar o BERT para extrair embedding de sentenças. 

### Importanto bibliotecas

In [1]:
import torch
from transformers import BertModel, BertTokenizer

### Tokenize

Modelos de deep learning usam tensores. Tensores são basicamente vetores e vetores são basicamente um conjunto de números. O tokenize é responsável por **transformar** o texto em um tipo de dado que pode ser **pré-processado** pelo modelo. Modelos podem apenas processarnúmeros, então os tokenizers precisam converter o input de texto para um dado número.

Existem vários algoritmos de tokenização, alguns deles são:

* **Word-based**: Separa o texto em palavras  encontra uma representação numérica pra cada uma delas.
* **Character-based**: Separa o texto em caracteres, em vez de palavras.
* **Subword tokenization**: Baseiam-se no princípio de que palavras usadas com frequência  não devem ser dividias em subpalavras menores, mas palavras raras devem ser decompostas em subpalavras significativas.

O BERT usa o algoritmo de tokenização chamado **WordPiece** que é baseado em **Subword tokenization**.

In [2]:
# Initialize the tokenizer with a pretrained model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Durando o pré-treinamento, o tokenizer foi "treinado" também, gerando um vocabulário conhecido. Cada palabra é associado a um index (um número) e essa número pode ser usado no modelo. Para lidar com palavras que o tokenizer não conhece ainda (out-of-vocabulary or OOV), uma técnica especial é usada para garantir que o tokenizer aprendeu "subword units". Isso significa que modelos pré-treinados não terão problemas de OOV. Quando o tokenizer não reconhece uma palavra (que não está no vocabulário) ele vai tentar separar essa palavra em partes pequenas que ele conhece. O tokenizer do BERT que usa WordPiece para separar os tokens. Exemplo: a palabra `granola` é separada em `gran` e `##ola` onde `##` indica o início da substring.

In [3]:
# Convert the string "granola bars" to tokenized vocabulary IDs
granola_ids = tokenizer.encode('granola bars')
# Print the IDs
print('granola_ids', granola_ids)
print('type of granola_ids', type(granola_ids))
# Convert the IDs to the actual vocabulary item
# Notice how the subword unit (suffix) starts with "##" to indicate 
# that it is part of the previous string
print('granola_tokens', tokenizer.convert_ids_to_tokens(granola_ids))

granola_ids [101, 12604, 6030, 6963, 102]
type of granola_ids <class 'list'>
granola_tokens ['[CLS]', 'gran', '##ola', 'bars', '[SEP]']


É possível perceber os "tokens espciais" [CLS] e [SEP]. Esses tokens são adicionados automaticamente pelo método `.encode()`. O primeiro é um token de classificação que já vem pré-treinado. É especificamente inserido para qualquer tipo de tarefa de classificação. Então, em vez de ter a média de todos os tokens e usar isso como representação da frase, é recomendado apenas pegar a saída do [CLS] que então representa a frase inteira. [SEP], por outro lado, é inserido como um separador entre várias instâncias. É usado como a previsão da próxima frase, onde é um separador entre a frase atual e a próxima. É especialmente importante lembrar que o toke [CLS] desempenha um grande papela nas tarefas de classificação e regressão.

### Tensores

Temos quase o o tipo certo de data para começar. Como vimos, o tipo de dado dos IDs de cada token é uma lista de inteiros. Vamos usar a biblioteca `transformers` junto com o PyTorch, que trabalha com tensores. Um tensor é um tipo especial de lista otimizada que normalmente é usada em deep learning. Para converter os IDs dos tokens em um tensor é só passar a lista de IDs no construtor do tensor. Aqui é usado um `LongTensor` que usado para inteiros. Para números de ponto flutuante é usado `FloatTensor` ou só `Tensor`. O método `.encode()` do tokenizer pode retornar um tensor ao em vez de uma lista passando o parâmetro `return_tensors='pt'`mas para ilustrar, vamos apenas fazer a conversão manualmente.

In [4]:
# Convert the list of IDs to a tensor of IDs 
granola_ids = torch.LongTensor(granola_ids)
# Print the IDs
print('granola_ids', granola_ids)
print('type of granola_ids', type(granola_ids))

granola_ids tensor([  101, 12604,  6030,  6963,   102])
type of granola_ids <class 'torch.Tensor'>


### O modelo

Agora que o input foi pré-processado em um tensor de IDs (lembrando que cada valor de ID corresponde ao ID do token no vocabulário criado pelo tokenizador). O modelo sabe qual palavra está sendo processada porque ele sabe qual token pertence a determinado ID. No BERT e na maioria dos modelos de linguagem baseados em Transformer, a primeira camada é uma camada de *embbeding*, cada token possu um *embedding* relacionado. No BERT, o *embedding* de um token é a soma de três tipos de *embeddings*: o *embedding* to token (gerado para o próprio *token*), o *embedding* do segmento (indica se o segmento faz parte da primeira ou da segunda sentença, não usado na inferência de uma única sentença) e o *embedding* de posição (distingue a posição do token na sentença).

Abaixo, uma imagem do BERT retirada do artigo publicado.

![enter image description here](https://github.com/BramVanroy/bert-for-inference/blob/master/img/bert-embeddings.png?raw=true)


#### Iniciando o modelo

Primeiramente é preciso iniciar o modelo, assim como o tokenizer, o modelo já é pré-treinado, o que  nos permite usar um modelo de linguagem já pré-treinado para obter representações de *token* ou de sentenças.

Vamos usar o mesmo modelo pré-treinado usado no tokenizer (`bert-base-uncased`). Ele é o menor modelo BERT que já foi treinado em textos com letras minúsculas. Como o modelo foi treinado com letras minúsculas, ele não sabe sobre textos com letras maiúsculas. O tokenizer automaticamente deixa o texto em letras minúsculas. Usar textos com letras minúsculas ou maiúsculas depende da tarefa. NER, por exemplo, podem requerer modelos treinados com maiúsculas e minúsculas (neste caso, troque "uncased" por "cased").

No exemplo abaixo, um argumento foi adicionado e passado para o iniciar o modelo. `output_hidden_states` dá informações de saída. Por padrão, o `BERTModel` vai retornar uma tupla mas o conteúdo dessa tupla é diferente dependendo da configuração do modelo. Quando passado `output_hidden_states=True`, a tupla vai conter:

1. O último estado oculto (`batch_size, sequence_length, hidden_size`)
2. `pooler_output` do token de classificação (`batch_size, hidden_size`)
3. os estados_ocultos das saídas do modelo em cada camada e as saídas dos embeddings iniciais (`batch_size, sequence_length, hidden_size`)

#### GPU x CPU

As placas gráficas (GPUs) são muito melhores em fazer operações em tensores do que uma CPU, portanto, sempre que disponível, executaremos os cálculos em GPU, como a CUDA (para isso, precisaremos de uma versão torch compatível com GPU.)

Assim, movemos nosso modelo para o dispositivo correto: se estiver disponível, moveremos o modelo `.to()` à GPU, caso contrário, permanecerá na CPU. É importante lembrar que o modelo e os dados a serem processados precisam estar no mesmo dispositivo.

Finalmente, definimos o modelo para o modo de avaliação (`.eval`), em contraste com o modo de treinamento (`.train()`). Na avaliação, não temos por exemplo o *dropout*.

In [5]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
# Set the device to GPU (cuda) if available, otherwise stick with CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = model.to(device)
granola_ids = granola_ids.to(device)

model.eval()

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [6]:
device

'cuda'

### Inferência

O modelo foi inicializado e a string de entrada ("granola_id") foi convertida em um tensor. Os modelos de linguagem (como o `BERTModel`, usado acima) possuem um método `foward()`, chamado automaticamente ao chamar o objeto. Esse método envia o tensor de entrada para frente do modelo e retorna a saída.

Como aqui trata-se de inferência, e não do treinamento ou ajuste (`fine-tuning`) do modelo, esta é a única etapa que chamamos o modelo esperando uma saída (`output`). Portanto, não precisamos otimizar o modleo para calcular gradientes e fazer o `backpropagation`.

Definimos `torch.no_grad()` na inferência para informar ao modelo que não faremos nenhum cálculo de gradiente e/ou retropropagação, tornando a inferência mais rápida e mais eficiente em termos de memória.

Geralmente, os métodos `model.eval()` e `torch.no_grad()` são usados juntos para avaliar e testar o modelo. Para treinar o modelos usamos o método `model.train()` e o método `torch.no_grad()` **não** deve ser usado.

### Lote (batch)

Abaixo, veremos um método chamado `.unsqueeze()`, que "descomprime" um tensor adicionando uma dimensão extra. Então, nosso tensor de granola de tamanho `(7,)` irá se transformar em um tensor de `(1, 7)`, onde `1` é a dimensão da frase. Essas suas dimensões são requeridas pelo modelo: ele é otimizado para treinar em **lotes** (batches), como veremos adiante.

Um lote consistem em vários textos de entrada "ao mesmo tempo" (geralmente em potêcia de dois, por exemplo, 64). Com um tamanho de lote de 64 (ou seja, 64 frases de uma vez), o tamanho do lote seria `(64, n)` onde `64` é o número de frases e `n` o comprimento da sequência. Aqui, onde usamos apenas uma entrada, isso não é importante, mas ao ajustar o modelo, precisamos trabalhar com lotes, pois o cálculo do gradiente será melhor para grandes lotes.

Nesses casos, `n` precisa ser o mesmo para todas as entradas, ou seja, nõa é possível ter uma sequência de 7 itens e uma de 12 itens (para lidar com isso, usamos técnicas de *paddind*). O tamanho de entrada do modelo precisa ser `(n_input_sentences, seq_len)` onde `seq_len` pode ser determinado de diferentes maneiras.

Duas escolhas populares são: usar o texto mais longo do lote como `seq_len` (por exemplo, 12) e preencher textos curtos até esse comprimento, ou definir um comprimento de sequência máximo fixo para o modelo (normalmente 512) e preencher todos os itens até este mesmo comprimento. A última abordagem é mais fácil de implementar, mas não é eficiente em termos de memória e é computacionalmente mais pesada. Fica a seu critério.

In [10]:
print(granola_ids.size())
# descomprimir IDs para obter o tamanho do lote = 1 como dimensão extra
granola_ids = granola_ids.unsqueeze(0)
print(granola_ids.size())

print(type(granola_ids))
with torch.no_grad():
    out = model(input_ids=granola_ids)

# a saída é uma tupla
print(type(out))
# a tupla contém três elementos, que serão explicados abaixo
print(len(out))
# aqui serão listados apenas os estados ocultos do modelo (hidden_states)
hidden_states = out[2]
##print(len(hidden_states))

torch.Size([5])
torch.Size([1, 5])
<class 'torch.Tensor'>
<class 'transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions'>
3


### Estado oculto (hidden state)

Como visto acima, nós enviamos os IDs de nossos tokens de entrada por meio do método `model()`, que chama internamente o método `foward()`. O `out` é uma tupla com todos os itens de saída relevantes, sendo o terceiro o mais importante, pois contém os estados ocultos (`hidden_states`) do modelo após a execução de um *foward*.

`hidden_states` é uma tupla de saída de cada camada no modelo para cada token. na execução anterior, vimos que cada tupla contém 13 itens. Quando você executa `print(model)`, a arquitetura do BertModel é exibida(todas as camadas, de cima pra baixo). O `hidden_states` inclui a saída da camada `embeddings` e a saída de todos os 12 `BERTLayer` no codificador. A saída de cada camada tem um tamanho de `(batch_size, sequence_length, 768)`.

Em nosso exemplo, isso é `(1, 7, 768)` porque temos apenas uma string de entrada (tamanho do lote = 1), e nossa string de entrada foi tokenizada em sete IDs (comprimento de sequência de 7). `768` é o número de dimensões ocultas.

Como podemos ver, há mais uma camada após o codificador, chamada pooler, que não faz parte dos hidden_states. Esta camada é usada para "agrupar" a saída do token de classificação. Sua saída é retornada no segundo item da tupla de saída out, conforme visto antes.

In [15]:
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

### Incorporação de sentença (sentence embeddings)

Agora que temos todos os `hidden_states`, podemos utilizá-lo em algumas tarefas. Por exemplo, para recuperar uma incorporação de frases (*sentence embeddings*) calculando a média de todos os tokens. Ou seja, vamos reduzir o tamanho de `(1, 7, 768)` para `(1, 768)` onde `1` é o tamanho do lote e `768` é o número de dimensões ocultas.

Há diversas maneiras de fazer uma abstração de frase de tokens, dependendo da tarefa de NLP. Aqui, estamos usando a média. Por enquanto, usaremos apenas a saída da última camada do codificador, isto é, hidden_states `[-1]`. É importante indicar que queremos pegar o torch.meansobre um determinado eixo. Uma vez que o tamanho da saída das camadas é `(1, 7, 768)`, queremos fazer a média sobre os sete tokens, que estão na segunda dimensão `(dim = 1)`.

In [16]:
sentence_embedding = torch.mean(hidden_states[-1], dim=1).squeeze()
print(sentence_embedding)
print(sentence_embedding.size())

tensor([ 2.7497e-01,  1.8313e-01, -8.8651e-02,  2.1698e-01,  3.1942e-01,
        -1.1412e-01,  7.4039e-02,  3.7655e-01, -4.1821e-01,  9.9971e-02,
        -9.0242e-02, -2.4298e-01,  1.5542e-01,  4.2042e-01, -2.5547e-01,
         2.9753e-01, -2.9643e-01, -2.5810e-02,  8.5306e-02,  1.0182e-01,
         3.0401e-01, -4.4263e-01,  3.1249e-02,  1.4435e-01,  3.0189e-01,
         7.3913e-02, -2.5580e-01,  3.1384e-01, -1.4688e-01, -1.5202e-01,
         7.0785e-02,  4.0448e-01, -1.1769e-01,  3.1848e-01,  2.8021e-02,
        -1.6934e-01,  3.5639e-01, -2.2931e-01, -1.1899e-01, -1.1182e-01,
        -1.6003e-01,  7.9355e-02,  5.1107e-01,  5.2223e-02, -1.5481e-01,
         2.8228e-02, -1.4365e-01, -4.7737e-01, -5.6638e-01, -4.8802e-01,
        -1.1429e-01,  2.8087e-01, -5.7160e-02,  2.3862e-01,  3.5440e-01,
         5.8237e-01,  1.2777e-01,  1.0363e-01,  3.0538e-01,  2.0989e-01,
         1.1693e-01,  2.6346e-01, -1.5832e-01, -1.1380e-01,  1.7189e-02,
        -3.4662e-02,  1.1470e-01,  3.2023e-02, -1.9

**Agora temos um vetor de 768 recursos que representam nossa sentença de entrada**. Mas podemos fazer mais! O artigo do BERT discute como alcançar os melhores resultados concatenando a saída das últimas quatro camadas.

Em nosso exemplo, isso significa que precisamos pegar as últimas quatro camadas de hidden_states, concatená-los e gerar a média. Nós queremos concatenar no eixo das dimensões ocultas de `768`. Como consequência, nosso vetor de saída concatenado irá ser do tamanho `(1, 7, 3072)` onde `3072 = 4 * 768`, ou seja, a concatenação de quatro camadas com uma dimensão oculta de 768. O vetor concatenado é muito maior do que a saída de apenas uma camada, o que significa que contém muito mais recursos.

Para algumas tarefas, esses recursos `3072` podem tem um desempenho melhor do que `768`.

Tendo um vetor de forma `(1, 7, 3072)`, ainda precisamos obter a média sobre a dimensão do token, como fizemos antes, ficando com um vetor de recurso de tamanho `(3072,)`.

In [17]:
# obter as ultimas quatro camadas
last_four_layers = [hidden_states[i] for i in (-1, -2, -3, -4)]
# juntas as camadas em uma tupla e concatenar com a ultima dimensão
cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
print(cat_hidden_states.size())

# pegar a média do vetor concatenado sobre a dimensão do token
cat_sentence_embedding = torch.mean(cat_hidden_states, dim=1).squeeze()
print(cat_sentence_embedding)
print(cat_sentence_embedding.size())

torch.Size([1, 5, 3072])
tensor([ 0.2750,  0.1831, -0.0887,  ...,  0.2894, -0.0034,  0.0764],
       device='cuda:0')
torch.Size([3072])


### Salvando e carregando resultados

É possível usar o vetor de recurso gerado em outro modelo ou tarefa, para isso basta salvar o tensor com `torch.save` e carregá-lo em outro script com `torch.load`, gerando arquivos na extensão .pt (PyTorch). Não é possível ler o arquivo salvo com um editor de texto (é um objeto especial que permite uma des(compressão) eficiente).

Também é possível salvar os tensores em um formato legível, convertendo em numpy e use algo como `np.savetxt` `('tensor.txt', your_tensor.numpy ())`, porém essa abordagem não é recomendada (é melhor usar o torch.save ou outra técnica de compressão).

Ao usar `.cpu ()`, dizemos ao PyTorch que queremos mover o tensor de saída de volta da GPU para a CPU. Isso não é obrigatório, mas é uma boa prática, ao fazer extração de recursos, mover os dados para a CPU. Desta forma, ao carregá-lo, ele é carregado como um tensor de CPU em vez de um tensor CUDA. Depois podemos mover novamente para a GPU, se necessário, mas usar a CPU por padrão é uma boa ideia (o tensor deve estar na CPU para podermos convertê-lo para `.numpy () ).

In [18]:
# sava nossa representação de sentença 
torch.save(cat_sentence_embedding.cpu(), 'my_sent_embed.pt')

# faz o load
loaded_tensor = torch.load('my_sent_embed.pt')
print(loaded_tensor)
print(loaded_tensor.size())

# converte para numpy para usar por ex com sklearn
np_loaded_tensor = loaded_tensor.numpy()
print(np_loaded_tensor)
print(type(np_loaded_tensor))

tensor([ 0.2750,  0.1831, -0.0887,  ...,  0.2894, -0.0034,  0.0764])
torch.Size([3072])
[ 0.2749734   0.1831336  -0.08865138 ...  0.28939357 -0.00340417
  0.07635488]
<class 'numpy.ndarray'>
